# [English version here!](./Get_Projects_Tiles_by_AOI_EN.ipynb)
# Déterminer les projets et les tuiles lidars couvrant une région d'intérêt

Ce tutoriel donne des exemples permettant de déterminer les projets et les tuiles lidars couvrant une région d'intérêt. Nous utilisons les fichiers d'index du produit [Nuages de points lidars - Série CanÉlévation](https://ouvert.canada.ca/data/fr/dataset/7069387e-9986-4297-9f55-0288e9676947). Principalement, nous utilisons la librairie [geopandas](https://geopandas.org/en/stable/) pour manipuler les couches d'index et faire les opérations spatiales.
Dans une application réelle, la liste des tuiles obtenues pour une région d'intérêt pourrait être utilisée pour construire une série de traitement plus avancé tel que décrit dans le notebook portant sur la [création d'un modèle numérique d'élévation (MNE) à partir d'un nuage de point lidar en format COPC LAZ](./DEM_from_COPC_lidar_FR.ipynb).

## Les étapes:
* [Préparation de l'environnement](#Pr%C3%A9paration-de-l'environnement)
* [Importation des modules nécessaires](#Importation-des-modules-n%C3%A9cessaires)
* [Téléchargement des index du produit](#T%C3%A9l%C3%A9chargement-des-index-du-produit)
* [Création des objets geopandas](#Cr%C3%A9ation-des-objets-geopandas)
* [Trouver les projets et les tuiles couvrant une zone d'intérêt](#Trouver-les-projets-et-les-tuiles-couvrant-une-zone-d'int%C3%A9r%C3%AAt)
* [Utilisation d'une région d'intérêt dans une autre projection](#Utilisation-d'une-r%C3%A9gion-d'int%C3%A9r%C3%AAt-dans-une-autre-projection)

## Préparation de l'environnement
Trois étapes sont nécessaires afin de pouvoir exécuter ces exemples localement.
### 1. Récupération du code source
   À l'endroit de votre choix sur votre ordinateur. Récupérer le code source des exemples en effectuant une commande de clone git.

   >```bash
   >git clone https://github.com/NRCan/CanElevation.git
   >```

   Aller ensuite dans le répertoire contenant les notebooks.

   >```bash
   >cd CanElevation/pointclouds_nuagespoints
   >```


### 2. Installation de conda

   Nous recommandons l'utilisation de conda pour l'installation des dépendances nécessaires au bon fonctionnement. Des [instructions d'installation détaillées](https://docs.anaconda.com/miniconda/install/#quick-command-line-install) sont disponibles. Les prochaines instructions assument que conda est disponible en ligne de commande.

### 3. Installation des dépendances

   Le fichier requirements.yaml contient les dépendances nécessaires. Nous devons donc créer un environnement conda avec ces dépendances.

   >```bash
   >conda env create -n CanElevation_PointClouds --file requirements.yml
   >```  

   On doit ensuite activer cet environnement conda nouvellement créé. 
	
   >```bash
   >conda activate CanElevation_PointClouds
   >```
    
Le lancement de jupyter notebook peut se faire simplement avec la commande suivante:

   >```bash
   >jupyter notebook
   >```
    
En cas de problème ou pour plus d'options de lancement, veuillez vous référer au [guide d'utilisation](https://jupyter-notebook-beginner-guide.readthedocs.io/en/latest/execute.html).

**Vous pouvez maintenant lancer le code des autres cellules du notebook**

## Importation des modules nécessaires
Dans les exemples suivants, nous utilisons principalement la librairie geopandas. Nous utilisons également le module requests permettant de télécharger les index ainsi que le module zipfile permettant de les décompresser localement.

In [3]:
import os
import requests
import zipfile
import geopandas as gpd
from shapely import box
from IPython.display import display
from ipywidgets import Output

## Téléchargement des index du produit
La [page du produit](https://open.canada.ca/data/en/dataset/7069387e-9986-4297-9f55-0288e9676947) contient les index des projets lidars distribués ainsi que les tuiles lidar composant chaque projet. Avant de pouvoir les manipuler, nous devons les télécharger et extraire le contenu des fichiers compressés.


In [4]:
# URLs to the shapefiles containing LiDAR projects and tiles index
# Liens URLs des index de projets et tuiles lidar
projects_url = 'https://download-telecharger.services.geo.ca/pub/elevation/pointclouds_nuagespoints/Index_LiDARprojects_projetslidar.zip'
tiles_url = 'https://download-telecharger.services.geo.ca/pub/elevation/pointclouds_nuagespoints/Index_LiDARtiles_tuileslidar.zip'

for index_url in (projects_url, tiles_url):
    zipname = os.path.basename(index_url)
    basename = os.path.splitext(zipname)[0]
    
    # Downloading / téléchargement
    response = requests.get(index_url)
    with open(zipname, 'wb') as file:
        file.write(response.content)

    #Unzipping / extraction
    with zipfile.ZipFile(zipname, 'r') as zip_ref:
        zip_ref.extractall(basename)
    

## Création des objets geopandas
Nous utiliserons la librairie Geopandas pour interagir avec les index. Cette librairie permet les opérations spatiales et géométriques. Veuillez consulter la documentation de la librairie pour plus d'informations.

In [5]:
# Loading the index in geopandas objects. The coordinate reference system of the index is NAD83(CSRS) - EPSG:4617
# Chargement des index dans des objets geopandas. Le système de référence de ces index est le NAD83(CSRS) - EPSG:4617
gdf_projects = gpd.read_file('Index_LiDARprojects_projetslidar/Index_LiDARprojects_projetslidar.shp')

# Load the lidar tiles / Chargement des tuiles lidars
gdf_tiles = gpd.read_file('Index_LiDARtiles_tuileslidar/Index_LiDARtiles_tuileslidar.shp')

## Trouver les projets et les tuiles couvrant une zone d'intérêt
Un cas d'usage commun est de déterminer les projets lidars et les tuiles couvrant une région d'intérêt. Dans cet exemple, nous définissons une zone d'intérêt. Nous déterminons ensuite les projets et les tuiles couvrant cette zone par une requête spatiale d'intersection. Les résultats sont imprimés à l'écran.

In [6]:

aoi = box(-80, 43.20, -79.971, 43.225)

aoi_gdf = gpd.GeoDataFrame({'geometry': [aoi]}, crs=4617)

overlapping_projects = gdf_projects[gdf_projects.intersects(aoi_gdf.geometry.iloc[0])]
overlapping_tiles = gdf_tiles[gdf_tiles.intersects(aoi_gdf.geometry.iloc[0])]

out = Output()
with out:
    print(f"Overlapping LiDAR Projects:")
    for _, row in overlapping_projects.iterrows():
        print(f"Project: {row['Project']}, URL: {row['URL']}")

    print(f"Overlapping LiDAR Tiles:")
    for _, row in overlapping_tiles.iterrows():
        print(f"Project: {row['Project']}, Tile name: {row['Tile_name']}")
display(out)

## Utilisation d'une région d'intérêt dans une autre projection
Travaillez-vous avec des coordonnées projetées? En connaissant le code de la projection, il est facile de les reprojeter avant de faire l'opération d'intersection sur les projets et les tuiles. 
Nous adaptons ici l'exemple précédent afin de trouver les projets et tuiles couvrant la zone d'intérêt projetée.

In [7]:
# AOI coordinates / Coordonnées de la région d'intérêt - NAD83(CSRS) / UTM zone 17N
aoi_utm = box(580736, 4783785, 583114, 4785855)

# We specify the UTM 17N EPSG code / Nous utilisons ici le code EPSG UTM zone 17N
aoi_gdf = gpd.GeoDataFrame({'geometry': [aoi_utm]}, crs=2958)

# The extra step needed is the reprojection of the AOI in 4617
# L'étape additionnelle requise est la reprojection de la zone d'intérêt en 4617
aoi_gdf = aoi_gdf.to_crs(epsg=4617)


# The remaining steps are the same / Les autres opérations sont inchangées
overlapping_projects = gdf_projects[gdf_projects.intersects(aoi_gdf.geometry.iloc[0])]
overlapping_tiles = gdf_tiles[gdf_tiles.intersects(aoi_gdf.geometry.iloc[0])]

out = Output()
with out:
    print(f"Overlapping LiDAR Projects:")
    for _, row in overlapping_projects.iterrows():
        print(f"Project: {row['Project']}, URL: {row['URL']}")

    print(f"Overlapping LiDAR Tiles:")
    for _, row in overlapping_tiles.iterrows():
        print(f"Project: {row['Project']}, Tile name: {row['Tile_name']}")
display(out)